In [1]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langdetect

In [3]:
pip install rake_nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np

# keyword extraction and clean up
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

#Text Blob for sentiment analysis
from textblob import TextBlob

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import linear_kernel

import string
import random
from PIL import Image
import requests
from io import BytesIO

from langdetect import detect

In [5]:
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"

In [6]:
hotel_df = pd.read_csv(url)

In [7]:
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive


In [8]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512470 entries, 0 to 512469
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Unnamed: 0                                  512470 non-null  int64  
 1   Hotel_Address                               512470 non-null  object 
 2   Additional_Number_of_Scoring                512470 non-null  int64  
 3   Review_Date                                 512470 non-null  object 
 4   Average_Score                               512470 non-null  float64
 5   Hotel_Name                                  512470 non-null  object 
 6   Reviewer_Nationality                        512470 non-null  object 
 7   Negative_Review                             512470 non-null  object 
 8   Review_Total_Negative_Word_Counts           512470 non-null  int64  
 9   Total_Number_of_Reviews                     512470 non-null  int64  
 

In [9]:
# combine reviews into one field
hotel_df["All_Reviews"] = hotel_df['Positive_Review'].astype(str) +"-"+ hotel_df["Negative_Review"]
hotel_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Positive,No real complaints the hotel was great great ...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Netherlands,Positive,Location was good and staff were ok It is cut...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,Netherlands,Negative,Great location in nice surroundings the bar a...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,Netherlands,Positive,Amazing location and building Romantic settin...


In [10]:
hotel_unique_df = hotel_df.drop_duplicates('Hotel_Name')
hotel_unique_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
405,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,18,2,9.6,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...
971,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,17,1,9.2,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",1 days,51.513734,-0.108751,UK,Positive,They have followed through special requests C...
2008,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,6,6,8.3,"[' Family with young children ', ' Triple Room...",1 days,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...
3778,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,231,1,9.2,"[' Leisure trip ', ' Group ', ' Comfort Double...",35 days,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...


In [11]:
hotel_unique_df.Hotel_Name.value_counts()

Hotel Arena                      1
Hotel 55                         1
Steigenberger Hotel Herrenhof    1
Hotel J ger                      1
Senator Hotel Vienna             1
                                ..
The Cleveland                    1
Grange Buckingham Hotel          1
Hilton London Olympia            1
H tel De Castiglione             1
Atlantis Hotel Vienna            1
Name: Hotel_Name, Length: 1475, dtype: int64

In [12]:
hotel_unique_df = hotel_unique_df.reset_index(drop=True)
hotel_unique_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...
1,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,18,2,9.6,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...
2,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,17,1,9.2,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",1 days,51.513734,-0.108751,UK,Positive,They have followed through special requests C...
3,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,6,6,8.3,"[' Family with young children ', ' Triple Room...",1 days,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...
4,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,231,1,9.2,"[' Leisure trip ', ' Group ', ' Comfort Double...",35 days,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...


In [13]:
#data pre-processing to remove stop words, punctuation, white space, and convert all words to lower case from combined reviews
hotel_unique_pp_df = hotel_unique_df.copy()
key_words = []
name_key_words = []

r = Rake()

for index, row in hotel_unique_pp_df.iterrows():
    r.extract_keywords_from_text(row['All_Reviews'])
    key_words_dict_scores = r.get_word_degrees()
    keywords = list(key_words_dict_scores.keys())
    
    key_words.append(keywords)

    r.extract_keywords_from_text(row['Hotel_Name'])
    key_words_dict_scores = r.get_word_degrees()
    name_keywords = list(key_words_dict_scores.keys())
    
    name_key_words.append(name_keywords)
    
hotel_unique_pp_df["key_words"] = key_words
hotel_unique_pp_df["name_key_words"] = name_key_words

hotel_unique_pp_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Reviewer_Score,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...,"[park, outside, hotel, beautiful, angry, made,...","[hotel, arena]"
1,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,9.6,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...,"[comfortable, beds, smart, bathroom, good, sho...","[k, hotel, george]"
2,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,9.2,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",1 days,51.513734,-0.108751,UK,Positive,They have followed through special requests C...,"[followed, special, requests, comfortable, bed...","[apex, temple, court, hotel]"
3,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,8.3,"[' Family with young children ', ' Triple Room...",1 days,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...,"[size, room, first, night, given, extremely, s...","[park, grand, london, paddington]"
4,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,9.2,"[' Leisure trip ', ' Group ', ' Comfort Double...",35 days,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...,"[nice, hotel, room, beautiful, bed, comfortabl...","[monhotel, lounge, spa]"


In [14]:
hotel_unique_pp_clean_df = hotel_unique_pp_df.copy()

In [15]:
# clean up keywords: drop symbols and numbers, as well as find root words

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


hotel_unique_pp_clean_df['clean_keywords']=hotel_unique_pp_clean_df['key_words'].map(lambda s:preprocess(s)) 
hotel_unique_pp_clean_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Tags,days_since_review,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...,"[park, outside, hotel, beautiful, angry, made,...","[hotel, arena]",park outside hotel beautiful angry made post a...
1,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,"[' Leisure trip ', ' Group ', ' Classic Twin R...",0 days,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...,"[comfortable, beds, smart, bathroom, good, sho...","[k, hotel, george]",comfortable beds smart bathroom good shower lo...
2,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,"[' Leisure trip ', ' Couple ', ' Deluxe King R...",1 days,51.513734,-0.108751,UK,Positive,They have followed through special requests C...,"[followed, special, requests, comfortable, bed...","[apex, temple, court, hotel]",followed special requests comfortable bed wide...
3,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,"[' Family with young children ', ' Triple Room...",1 days,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...,"[size, room, first, night, given, extremely, s...","[park, grand, london, paddington]",size room first night given extremely small ba...
4,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,"[' Leisure trip ', ' Group ', ' Comfort Double...",35 days,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...,"[nice, hotel, room, beautiful, bed, comfortabl...","[monhotel, lounge, spa]",nice hotel room beautiful bed comfortable expe...


In [16]:
hotel_sub_df = hotel_unique_pp_clean_df.copy()

In [17]:
# run text blob in a loop

polarities = []
subjectivities = []

for index, row in hotel_sub_df.iterrows():
    sentence = row['All_Reviews']
    analysis = TextBlob(sentence).sentiment
    polarity = analysis[0]
    subjectivity = analysis[1]

    # populate lists
    polarities.append(polarity)
    subjectivities.append(subjectivity)


hotel_sub_df['sentiment_polarity'] = polarities
hotel_sub_df['sentiment_subjectivity']= subjectivities

hotel_sub_df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,lat,lng,Hotel_Country,Sentiment,All_Reviews,key_words,name_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,52.360576,4.915968,Netherlands,Negative,Only the park outside of the hotel was beauti...,"[park, outside, hotel, beautiful, angry, made,...","[hotel, arena]",park outside hotel beautiful angry made post a...,0.046438,0.452243
1,405,1 15 Templeton Place Earl s Court Kensington a...,244,2017-08-03,8.5,K K Hotel George,Ireland,Really nothing negative to say about this hotel,9,1831,...,51.491888,-0.194971,UK,Positive,Very comfortable beds smart bathroom good sho...,"[comfortable, beds, smart, bathroom, good, sho...","[k, hotel, george]",comfortable beds smart bathroom good shower lo...,0.267698,0.591799
2,971,1 2 Serjeant s Inn Fleet Street City of London...,639,2017-08-02,9.2,Apex Temple Court Hotel,United Kingdom,Difficult to locate the phone to contact rece...,20,2619,...,51.513734,-0.108751,UK,Positive,They have followed through special requests C...,"[followed, special, requests, comfortable, bed...","[apex, temple, court, hotel]",followed special requests comfortable bed wide...,0.014354,0.668776
3,2008,1 3 Queens Garden Westminster Borough London W...,1058,2017-08-02,7.7,The Park Grand London Paddington,Cyprus,The first night we were given an extremely sm...,30,4380,...,51.514218,-0.180903,UK,Positive,The size of the room- The first night we were...,"[size, room, first, night, given, extremely, s...","[park, grand, london, paddington]",size room first night given extremely small ba...,0.050000,0.388889
4,3778,1 3 Rue d Argentine 16th arr 75116 Paris France,26,2017-06-29,8.4,Monhotel Lounge SPA,Brazil,No Negative,0,171,...,48.874348,2.289733,France,Positive,Nice hotel Room was beautiful and bed very co...,"[nice, hotel, room, beautiful, bed, comfortabl...","[monhotel, lounge, spa]",nice hotel room beautiful bed comfortable expe...,0.259738,0.450035


In [18]:
hotel_sub_df.Hotel_Name.value_counts()

Hotel Arena                      1
Hotel 55                         1
Steigenberger Hotel Herrenhof    1
Hotel J ger                      1
Senator Hotel Vienna             1
                                ..
The Cleveland                    1
Grange Buckingham Hotel          1
Hilton London Olympia            1
H tel De Castiglione             1
Atlantis Hotel Vienna            1
Name: Hotel_Name, Length: 1475, dtype: int64

In [19]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [20]:
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

In [21]:
tfidf_matrix

matrix([[0.        , 0.        , 0.07687068, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [22]:
#calculate cosine matrix
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

C:\Users\16513\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


[[1.         0.02775123 0.         ... 0.04576608 0.01198679 0.01373626]
 [0.02775123 1.         0.01933517 ... 0.00948075 0.         0.0231052 ]
 [0.         0.01933517 1.         ... 0.         0.         0.        ]
 ...
 [0.04576608 0.00948075 0.         ... 1.         0.         0.00624459]
 [0.01198679 0.         0.         ... 0.         1.         0.00428577]
 [0.01373626 0.0231052  0.         ... 0.00624459 0.00428577 1.        ]]


In [23]:
indices = pd.Series(hotel_sub_df['Hotel_Name'])

In [24]:
#get recommendations based on tf-idf and cosine_similarities (hotel name)

def recommend_name(Hotel_Name, cosine_similarities = cosine_similarities):
    
    recommended_hotel = []
    recommended_hotel_address = []
    recommended_hotel_rating = []
    recommended_hotel_score = []
         
    idx = indices[indices == Hotel_Name].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
        recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
        recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
        recommended_hotel_score.append(score_series[i])
        
    data = {'Hotel': recommended_hotel,
           'Address': recommended_hotel_address,
            'Rating': recommended_hotel_rating,
            'Score': recommended_hotel_score
                       }
    
    rec_df = pd.DataFrame(data)
    
    return rec_df

In [25]:
rec_filter_df = recommend_name('Copthorne Tara Hotel London Kensington')
rec_filter_df = rec_filter_df[(rec_filter_df['Rating'] > 8)]
rec_filter_df.head()

,Hotel,Address,Rating,Score
0,The Chamberlain,130 135 Minories City of London London EC3N 1N...,8.9,0.207321
1,Caesar Hotel,26 33 Queens Gardens Westminster Borough Londo...,8.2,0.179883
2,Primero Primera,Doctor Carulla 25 Sarri St Gervasi 08017 Barce...,9.2,0.172148
3,The Cavendish London,81 Jermyn Street Westminster Borough London SW...,8.8,0.158376
4,The Westbourne Hyde Park,51 Gloucester Terrace Westminster Borough Lond...,8.9,0.153060


In [26]:
# prepping for Flask

In [27]:
#code that stays consistent
url= "https://ralston-hotel-data-bucket.s3.amazonaws.com/Hotel_Reviews_Clean.csv"
hotel_df = pd.read_csv(url)

tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)
tfidf_matrix = tfidf.fit_transform(hotel_sub_df['clean_keywords']).todense()

cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

C:\Users\16513\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [28]:
np.save("hotel_cos_data.npy", cosine_similarities)

In [29]:
hotel_arr = np.load("hotel_cos_data.npy")

In [30]:
print(hotel_arr[0:5])

[[1.         0.02775123 0.         ... 0.04576608 0.01198679 0.01373626]
 [0.02775123 1.         0.01933517 ... 0.00948075 0.         0.0231052 ]
 [0.         0.01933517 1.         ... 0.         0.         0.        ]
 [0.03748136 0.03275728 0.         ... 0.04752206 0.00767096 0.0059227 ]
 [0.06073488 0.07644466 0.04567874 ... 0.05012176 0.04066727 0.03397718]]


In [31]:
#uploaded .npy arr file to S3 bucket
hotel_arr_url = "https://ralston-hotel-data-bucket.s3.amazonaws.com/hotel_cos_data.npy"

In [32]:
pip install s3fs

Note: you may need to restart the kernel to use updated packages.


In [33]:
import s3fs

In [34]:
s3 = s3fs.S3FileSystem(anon=True)

In [35]:
hotel_arr_s3 = np.load(s3.open("ralston-hotel-data-bucket/hotel_cos_data.npy"))

In [36]:
print(hotel_arr_s3[0:5])

[[1.         0.02775123 0.         ... 0.04576608 0.01198679 0.01373626]
 [0.02775123 1.         0.01933517 ... 0.00948075 0.         0.0231052 ]
 [0.         0.01933517 1.         ... 0.         0.         0.        ]
 [0.03748136 0.03275728 0.         ... 0.04752206 0.00767096 0.0059227 ]
 [0.06073488 0.07644466 0.04567874 ... 0.05012176 0.04066727 0.03397718]]


In [37]:
#example variable
Hotel_Name = "Primero Primera"


#variable reliant code
indices = pd.Series(hotel_sub_df['Hotel_Name'])

recommended_hotel = []
recommended_hotel_address = []
recommended_hotel_rating = []
recommended_hotel_score = []

idx = indices[indices == Hotel_Name].index[0]

score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

top_10_indices = list(score_series.iloc[1:11].index)

for i in top_10_indices:
    recommended_hotel.append(list(hotel_sub_df['Hotel_Name'])[i])
    recommended_hotel_address.append(list(hotel_sub_df['Hotel_Address'])[i])
    recommended_hotel_rating.append(list(hotel_sub_df['Average_Score'])[i])
    recommended_hotel_score.append(score_series[i])
    

data = {'Hotel': recommended_hotel,
        'Address': recommended_hotel_address,
        'Rating': recommended_hotel_rating,
        'Score': recommended_hotel_score
        }

rec_df = pd.DataFrame(data)
rec_df = rec_df[(rec_df['Rating'] > 5)]

rec_df


,Hotel,Address,Rating,Score
0,Crowne Plaza Amsterdam South,George Gershwinlaan 101 Zuideramstel 1082 MT A...,9.0,0.312552
1,Ramada Plaza Milano,Via Stamira D Ancona 27 Distretto Viale Monza ...,8.4,0.243432
2,Club Quarters Hotel Trafalgar Square,8 Northumberland Avenue Westminster Borough Lo...,8.5,0.242138
3,Bassano,15 Rue de Bassano 16th arr 75016 Paris France,8.3,0.219235
4,Le Parisis Paris Tour Eiffel,102 Bd De Grenelle 15th arr 75015 Paris France,8.6,0.206636
5,Amp re,102 Avenue de Villiers 17th arr 75017 Paris Fr...,8.8,0.204313
6,Hotel Spa Villa Olimpica Suites,Pallars 121 125 Sant Mart 08018 Barcelona Spain,8.0,0.200575
7,Crowne Plaza London Ealing,Western Avenue Ealing London W5 1HG UK,8.5,0.199171
8,Corendon Vitality Hotel Amsterdam,Aletta Jacobslaan 7 Slotervaart 1066 BP Amster...,8.4,0.199009
9,Le Narcisse Blanc Spa,19 Boulevard De La Tour Maubourg 7th arr 75007...,9.5,0.195490
